<a href="https://colab.research.google.com/github/stef4k/train-maintenance-data-mining/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification


In [76]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from collections import Counter
import ast
import pickle
from joblib import Parallel, delayed
from google.colab import files
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from gensim.models import Word2Vec
from scipy.optimize import minimize
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tqdm import tqdm
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")


Manually remove the first ';' from the first row in csv file

In [77]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';', index_col=0)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
373,4451319,"[541, 541, 541, 541, 541, 541, 541, 541, 541, ...","[3636, 2956, 2956, 2956, 2956, 2956, 4066, 363...","[-14390, -14306, -14294, -14273, -14269, -1424...",50.948527,4.417702,"[0.0, 88.7, 90.2, 80.4, 78.6, 57.2, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",99
991,4611315,"[547, 547, 547, 547, 547, 547, 547, 547, 547, ...","[4068, 4068, 4068, 4068, 4068, 3658, 4068, 365...","[-14273, -14006, -13798, -13647, -13502, -1347...",50.558296,3.919084,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.0, 0.0, 0.1, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",9


In [42]:
class Word2VecVectorizer(TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.w2v_model = None

    def fit(self, X, y=None):
        sentences = [sentence.split() for sentence in X]
        self.w2v_model = Word2Vec(sentences, vector_size=self.size, window=self.window,
                                  min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X, y=None):
        transformed_data = np.array([
            np.mean([self.w2v_model.wv[word] for word in sentence.split() if word in self.w2v_model.wv]
                    or [np.zeros(self.
                                 size)], axis=0)
            for sentence in X
        ])
        return csr_matrix(transformed_data)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [43]:
class EnsembleModel:
    def __init__(self, trained_models, trained_vectorizers):

        self.trained_models = trained_models
        self.trained_vectorizers = trained_vectorizers
        self.optimized_weights = None

    def _generate_prediction_matrix(self, X):
        predictions = {}
        for (vect_name, samp_name, clf_name), model in self.trained_models.items():
            vectorizer = deepcopy(self.trained_vectorizers[vect_name])
            X_vect = vectorizer.transform(X).toarray()

            predictions[(vect_name, samp_name, clf_name)] = model.predict_proba(X_vect)

        return np.stack(list(predictions.values()), axis=2)

    def fit(self, X_train, y_train):
        pred_matrix = self._generate_prediction_matrix(X_train)

        def fitness(weights):
            weighted_pred = np.tensordot(pred_matrix, weights, axes=([2], [0]))
            final_pred = np.argmax(weighted_pred, axis=1)
            return -f1_score(y_train, final_pred, average='weighted')

        num_models = pred_matrix.shape[2]
        bounds = [(0, 1)] * num_models
        result = differential_evolution(fitness, bounds)

        self.optimized_weights = result.x

    def predict(self, X):
        if self.optimized_weights is None:
            raise ValueError("The ensemble model must be trained using `train_ensemble` before prediction.")

        pred_matrix = self._generate_prediction_matrix(X)
        weighted_pred = np.tensordot(pred_matrix, self.optimized_weights, axes=([2], [0]))
        ensemble_pred = np.argmax(weighted_pred, axis=1)
        return ensemble_pred


In [78]:
class Representations:
  def __init__(self, df):
    self.df = df

  def representation_a(self, df):
            events_types_dict = {}
            for events_sequence in df['events_sequence']:
                row_list = ast.literal_eval(events_sequence)
                unique_events = set(row_list)
                for event in unique_events:
                    if not events_types_dict.get(event):
                        events_types_dict[event] = 0
                    events_types_dict[event] += 1

            sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
            sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
            sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / df.shape[0] * 100
            sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)

            events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage <= 85].event_type)))

            df['clean_events_sequence'] = (
                df['events_sequence']
                .apply(ast.literal_eval)
                .apply(lambda x: [i for i in x if i in events_low_frequency])
                .astype(str)
                .replace(r'[\[\],]', '', regex=True)
            )

            self.y = df['incident_type'].copy()
            self.X = df['clean_events_sequence'].copy()
            return self.X, self.y

  def representation_b(self, df):

    before_incident = []
    after_incident = []

    for _, row in df.iterrows():
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        before_events = " ".join([str(event) for event, time in zip(events, seconds) if time <= 0])
        if before_events:
            before_incident.append({
                "events_sequence": before_events,
                "class": incident_type
            })

        after_events = " ".join([str(event) for event, time in zip(events, seconds) if time > 0])
        if after_events:
            after_incident.append({
                "events_sequence": after_events,
                "class": 100
            })

    before_df = pd.DataFrame(before_incident)
    after_df = pd.DataFrame(after_incident)

    return before_df, after_df


  def representation_c(self, df, sequence_length=30):

    overlapping_sequences = []
    step = sequence_length // 2

    for _, row in df.iterrows():
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        for i in range(0, len(events) - sequence_length + 1, step):
            sequence = [str(i) for i in events[i:i + sequence_length]]
            seconds_slice = seconds[i:i + sequence_length]
            sequence_class = incident_type

            overlapping_sequences.append({"sequence": " ".join(sequence), "class": sequence_class})

    sequences_df = pd.DataFrame(overlapping_sequences)
    return sequences_df.sequence, sequences_df['class']


In [59]:

class Experiment:
    def __init__(self, X, y):
        self.X = X
        self.le = LabelEncoder()
        self.y = self.le.fit_transform(y)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
        self.y_train = self.le.transform(self.y_train)
        self.y_test = self.le.transform(self.y_test)
        self.trained_models = {}
        self.trained_vectorizers = {}
        self.trainer_samplers = {}
        self.results = []
        self.sampling_strategies = {
            "SMOTE": SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            "Borderline-SMOTE": BorderlineSMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            "ADASYN": ADASYN(sampling_strategy='auto', random_state=1, n_neighbors=3),
            "RandomOversampler": RandomOverSampler(sampling_strategy='auto', random_state=1),
            "SMOTE-ENN": SMOTEENN(sampling_strategy='auto', random_state=1),
            "SMOTE-Tomek": SMOTETomek(sampling_strategy='auto', random_state=1)
        }

        self.vectorizers = {
            "TFIDF": TfidfVectorizer(),
            "Count": CountVectorizer(),
            "Word2Vec": Word2VecVectorizer(size=100, window=5, min_count=1)
        }
        self.classifiers = {
            'LogisticRegression': LogisticRegression(),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GaussianNB': GaussianNB(),
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(probability=True),
            'XGBoost': XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False, n_jobs = 2),
        }


    def duplicate_minor_classes(self, X, y, min_instances=5):
        X = X.reset_index(drop=True)
        y  = y.reset_index(drop=True)
        class_counts = y.value_counts()

        minor_classes = class_counts[class_counts < min_instances].index

        minor_class_rows = X.loc[y.isin(minor_classes)]
        minor_class_labels = y.loc[y.isin(minor_classes)]

        duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
        duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

        X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
        y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

        X_balanced.reset_index(drop=True, inplace=True)
        y_balanced.reset_index(drop=True, inplace=True)

        return X_balanced, y_balanced


    def test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name):
        """Test a model with Stratified K-Fold and return an array with metric results."""
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        accuracies, recalls, precisions, f1s = [], [], [], []
        X, y = self.X_train, self.y_train

        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            X_train = vectorizer.fit_transform(X_train).toarray()
            X_test = vectorizer.transform(X_test).toarray()

            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

            model.fit(X_resampled, y_resampled)

            y_pred = model.predict(X_test)

            accuracies.append(accuracy_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred, average='weighted'))
            precisions.append(precision_score(y_test, y_pred, average='weighted'))
            f1s.append(f1_score(y_test, y_pred, average='weighted'))

        return [
            model_name, vectorizer_name, sampler_name,
            np.mean(accuracies), np.std(accuracies),
            np.mean(recalls), np.std(recalls),
            np.mean(precisions), np.std(precisions),
            np.mean(f1s), np.std(f1s)
        ]

    def training(self):
        results = []
        progress_bar = tqdm(total = len(self.vectorizers.keys()) * len(self.sampling_strategies.keys()) * len(self.classifiers.keys()))
        self.X_train, self.y_train = self.duplicate_minor_classes(self.X_train, pd.Series(self.y_train))
        for vect_name, ovectorizer in self.vectorizers.items():
            vectorizer = deepcopy(ovectorizer)
            for samp_name, osampler in self.sampling_strategies.items():
                sampler = deepcopy(osampler)
                for clf_name, omodel in self.classifiers.items():
                    model = deepcopy(omodel)

                    result = self.test(
                        model=model,
                        model_name=clf_name,
                        vectorizer=vectorizer,
                        vectorizer_name=vect_name,
                        sampler=sampler,
                        sampler_name=samp_name
                    )
                    results.append(result)

                    vectorizer = deepcopy(ovectorizer)
                    sampler = deepcopy(osampler)
                    model = deepcopy(omodel)
                    vectorizer.fit(self.X_train)
                    X_train = vectorizer.transform(self.X_train).toarray()
                    X_resampled, y_resampled = sampler.fit_resample(X_train, self.y_train)
                    model.fit(X_resampled, y_resampled)
                    self.trained_models[(vect_name, samp_name, clf_name)] = model
                    self.trained_vectorizers[vect_name] = vectorizer
                    self.trainer_samplers[(vect_name, samp_name)] = sampler
                    progress_bar.update(1)

        columns = [
            'Model', 'Vectorizer', 'Sampler',
            'Accuracy Mean', 'Accuracy Std',
            'Recall Mean', 'Recall Std',
            'Precision Mean', 'Precision Std',
            'F1 Mean', 'F1 Std'
        ]
        results_df = pd.DataFrame(results, columns=columns)
        self.results = results_df
        return results_df




In [60]:
class Evolution:
    def __init__(self, X, y):

        self.original_X = X
        self.original_y = y
        self.sequences = [x.split() for x in X]
        self.labels = y
        self.keep = self.get_unique_events()
        self.remove = []
        self.candidates = self.get_candidates()
        self.next = self.candidates[0] if self.candidates else None
        self.best_score = None

    def get_unique_events(self):
        return list(set(event for seq in self.sequences for event in seq))

    def get_candidates(self):
        sequences_flat = [(i, event) for i, seq in enumerate(self.sequences) for event in seq]
        df = pd.DataFrame(sequences_flat, columns=["seq_idx", "event"])
        total_occurrences = df.shape[0]

        class_counts = df.groupby("event").apply(
            lambda x: len(set(self.labels[x["seq_idx"].values]))
        )
        event_counts = df["event"].value_counts()
        gamma_scores = (class_counts / len(set(self.labels))) * (event_counts / total_occurrences)

        return gamma_scores.sort_values(ascending=False).index.tolist()

    def calculate_fitness(self):
        mutated_X = [' '.join(seq) for seq in self.sequences]
        mutated_y = self.labels

        skf = StratifiedKFold(n_splits=5)
        f1_scores = []

        def fit_model(train_idx, test_idx):
            X_train = [mutated_X[i] for i in train_idx]
            X_test = [mutated_X[i] for i in test_idx]
            y_train, y_test = mutated_y[train_idx], mutated_y[test_idx]

            vectorizer = TfidfVectorizer()
            X_train_transformed = vectorizer.fit_transform(X_train).toarray()
            X_test_transformed = vectorizer.transform(X_test).toarray()

            smote = SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3)
            X_resampled, y_resampled = smote.fit_resample(X_train_transformed, y_train)

            model = XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False, n_jobs = 2)
            model.fit(X_resampled, y_resampled)

            y_pred = model.predict(X_test_transformed)
            return f1_score(y_test, y_pred, average="weighted")

        f1_scores = Parallel(n_jobs=-1)(
            delayed(fit_model)(train_idx, test_idx) for train_idx, test_idx in skf.split(mutated_X, mutated_y)
        )
        mean_f1 = np.mean(f1_scores)
        std_f1 = np.std(f1_scores)
        return mean_f1 - std_f1

    def mutate(self):
        if self.best_score is None:
            self.best_score = self.calculate_fitness()
            print("Starting score:", self.best_score)
            return

        modified_sequences = [[event for event in seq if event != self.next] for seq in self.sequences]

        original_sequences = self.sequences[:]

        self.sequences = modified_sequences
        fitness_score = self.calculate_fitness()

        if fitness_score >= self.best_score:
            self.remove.append(self.next)
            self.keep.remove(self.next)
            self.best_score = fitness_score
            print("New best score:", self.best_score)
        elif self.best_score - fitness_score <= 0.03:
            self.sequences = [seq for seq in modified_sequences if seq]
            self.remove.append(self.next)
            self.keep.remove(self.next)
            print("Noise removed", fitness_score)
        else:
            self.sequences = original_sequences
            print("No changes", fitness_score)

        self.candidates = [e for e in self.candidates if e not in self.remove]
        self.next = self.candidates[0] if self.candidates else None

    def save_state(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        print(f"State saved to {filename}.")

    @classmethod
    def load_state(cls, filename):
        with open(filename, 'rb') as f:
            obj = pickle.load(f)
        print(f"State loaded from {filename}.")
        return obj

    def run(self, generations):
        c = 0
        for _ in tqdm(range(generations)):
            if not self.next:
                break
            self.mutate()
            if c == 30:
              self.save_state("evolution_state.pkl")
              files.download('evolution_state.pkl')
              c = 0
            c += 1


In [61]:
def duplicate_minor_classes(X, y, min_instances=10):
    X = X.reset_index(drop=True)
    y  = y.reset_index(drop=True)
    class_counts = y.value_counts()

    minor_classes = class_counts[class_counts <= min_instances].index

    minor_class_rows = X.loc[y.isin(minor_classes)]
    minor_class_labels = y.loc[y.isin(minor_classes)]

    duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
    duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

    X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
    y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

    X_balanced.reset_index(drop=True, inplace=True)
    y_balanced.reset_index(drop=True, inplace=True)

    return X_balanced, y_balanced

Testing the pipeline with no sequence modification:

In [ ]:
X, y = duplicate_minor_classes(df, df.incident_type)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

100%|██████████| 180/180 [4:34:36<00:00, 91.54s/it]


              precision    recall  f1-score   support

           0       0.74      0.83      0.78        24
           1       1.00      1.00      1.00         3
           2       0.92      0.69      0.79        16
           3       0.50      0.25      0.33         4
           4       0.67      1.00      0.80         2
           5       0.62      0.78      0.69        23
           6       0.00      0.00      0.00         5
           7       0.88      0.80      0.84        64
           8       0.70      0.53      0.60        30
           9       0.67      1.00      0.80         2
          10       0.75      1.00      0.86         6
          11       0.51      0.66      0.57        35

    accuracy                           0.71       214
   macro avg       0.66      0.71      0.67       214
weighted avg       0.72      0.71      0.71       214



In [ ]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
64,GradientBoostingClassifier,Count,SMOTE,0.718129,0.050224,0.718129,0.050224,0.722637,0.054595,0.712874,0.053408
74,GradientBoostingClassifier,Count,Borderline-SMOTE,0.711111,0.039247,0.711111,0.039247,0.720117,0.038706,0.707874,0.040774
84,GradientBoostingClassifier,Count,ADASYN,0.712281,0.050360,0.712281,0.050360,0.717498,0.053334,0.707082,0.052112
114,GradientBoostingClassifier,Count,SMOTE-Tomek,0.702924,0.046549,0.702924,0.046549,0.707647,0.050819,0.697597,0.048697
94,GradientBoostingClassifier,Count,RandomOversampler,0.695906,0.034696,0.695906,0.034696,0.716885,0.038906,0.695975,0.038387
54,GradientBoostingClassifier,TFIDF,SMOTE-Tomek,0.683041,0.044598,0.683041,0.044598,0.727463,0.046621,0.687537,0.046580
34,GradientBoostingClassifier,TFIDF,RandomOversampler,0.684211,0.024253,0.684211,0.024253,0.702683,0.025199,0.684024,0.024986
4,GradientBoostingClassifier,TFIDF,SMOTE,0.676023,0.026824,0.676023,0.026824,0.711289,0.030520,0.678414,0.028567
24,GradientBoostingClassifier,TFIDF,ADASYN,0.677193,0.027529,0.677193,0.027529,0.707741,0.027595,0.678114,0.029618
14,GradientBoostingClassifier,TFIDF,Borderline-SMOTE,0.678363,0.035859,0.678363,0.035859,0.708378,0.030483,0.677870,0.038310


In [ ]:
X, y = Representations(df).representation_a(df)
temp = pd.concat([X, y], axis=1)
X, y = duplicate_minor_classes(temp.clean_events_sequence, temp.incident_type)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

 31%|███       | 56/180 [1:40:43<7:23:11, 214.45s/it] 

In [ ]:
results

In [ ]:
df_before, df_after = Representations(df).representation_b(df)
X, y = pd.concat([df_before.events_sequence, df_after.events_sequence]), pd.concat([df_before['class'], df_after['class']])
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

In [ ]:
results

In [ ]:
X, y = Representations(df).representation_c(df, sequence_length=100)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

In [ ]:
results